### Build a function to wrangle data

In [1]:
import pandas as pd

In [2]:

def wrangle(filepath):
    # Import 
    df = pd.read_csv(filepath)

    # Check if the price column contains "B"
    contains_B = df["price"].str.contains("B")
    if contains_B.any(): # Check if there's a True in the list 
        df.loc[contains_B, "price"] = (df.loc[contains_B, "price"].str.replace(",", "").str.replace("B", "")).astype(float)*1000
        df.loc[contains_B, "price"] = df.loc[contains_B, "price"].astype(str) + "M"
    
    # Check if the price column does not contain "M"
    contains_M = df["price"].str.contains("M")
    if not contains_M.all():
        # Divide the values by 1 million, then add "M"
        df.loc[~contains_M, "price"] = df.loc[~contains_M, "price"].str.replace(",", "").astype(float)/1e6
        df.loc[~contains_M, "price"] = df.loc[~contains_M, "price"].astype(str) + "M"
    
    # Ensure all values in the price column are numeric 
    df["price"] = df["price"].str.replace("M", "").str.replace(",", "").astype(float)

    # Convert the price back to millions 
    df["price"] = df["price"] * 1e6

    # Remove "ft²" and "," from "usable_area" and "gross_floor_area" and convert to float
    df["usable_area"] = df["usable_area"].str.replace("ft²", "").str.replace(",", "").astype(float)
    df["gross_floor_area"] = df["gross_floor_area"].str.replace("ft²", "").str.replace(",", "").astype(float)

    # Create "price_per_sq_foot" columns
    df["price_per_sq_foot_SA"] = df["price"] / df["usable_area"]
    df["price_per_sq_foot_GFA"] = df["price"] / df["gross_floor_area"]

    # Sort df by price per square foot using S.A. in ascending order, in case of ties, sort by GFA
    # Reset the row index
    df = df.sort_values(by=["price_per_sq_foot_SA", "price_per_sq_foot_GFA"], ascending=True).reset_index(drop=True)
    
    return df

In [3]:
df = wrangle("data/hk-centaline-property-buy.csv")
df.head()

,name,room_numbers,address,price,usable_area,gross_floor_area,url,latitude,longitude,price_per_sq_foot_SA,price_per_sq_foot_GFA
0,KING LAM ESTATE・KING LUI HOUSE (BLOCK 2),Low Floor・FLAT 21・1 Room,Po Lam,1400000.0,349.0,NaN,https://hk.centanet.com/findproperty/en/detail...,22.323713,114.254880,4011.461318,NaN
1,TAK TIN ESTATE・TAK YEE HOUSE (BLOCK 3),(1 Suite),Lam Tin,1800000.0,443.0,NaN,https://hk.centanet.com/findproperty/en/detail...,22.308020,114.237564,4063.205418,NaN
2,FU SHIN ESTATE・SHIN KWAN HOUSE (BLOCK 1),1 Room,Tai Po Town Centre,1200000.0,294.0,NaN,https://hk.centanet.com/findproperty/en/detail...,22.442322,114.165506,4081.632653,NaN
3,Long Ping Estate・HOR PING HOUSE,Low Floor・FLAT 35・2 Rooms,Long Ping,1480000.0,355.0,NaN,https://hk.centanet.com/findproperty/en/detail...,NaN,NaN,4169.014085,NaN
4,TAI WO ESTATE・OI WO HOUSE (BLOCK 2),Studio,Tai Po Town Centre,1480000.0,349.0,NaN,https://hk.centanet.com/findproperty/en/detail...,22.442322,114.165506,4240.687679,NaN


In [4]:
# Most expensive per square foot house url: 
print(f"The most expensive per square foot house url: {df.loc[9999,"url"]}")

# Least expensive per square foot house url:
print(f"The least expensive per square foot house url: {df.loc[0,"url"]}")

The most expensive per square foot house url: https://hk.centanet.com/findproperty/en/detail/BEL-AIR-RISE_TLA343?theme=buy
The least expensive per square foot house url: https://hk.centanet.com/findproperty/en/detail/KING-LAM-ESTATE-KING-LUI-HOUSE-(BLOCK-2)_UDU107?theme=buy


In [5]:
df.info()
df.shape

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 11 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   name                   10000 non-null  object 
 1   room_numbers           9985 non-null   object 
 2   address                9996 non-null   object 
 3   price                  10000 non-null  float64
 4   usable_area            10000 non-null  float64
 5   gross_floor_area       6158 non-null   float64
 6   url                    10000 non-null  object 
 7   latitude               8591 non-null   float64
 8   longitude              8591 non-null   float64
 9   price_per_sq_foot_SA   10000 non-null  float64
 10  price_per_sq_foot_GFA  6158 non-null   float64
dtypes: float64(7), object(4)
memory usage: 859.5+ KB


(10000, 11)

In [6]:
# Save the cleaned df
df.to_csv("data/hk-centaline-property-buy-clean.csv", index=False)